In [20]:
from transformers import AutoTokenizer, AutoModel
import torch
import os
from info_nce import InfoNCE, info_nce
model_name = 'BAAI/bge-m3'
BATCH_SIZE=1
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
def worker_init_fn(worker_id):
    seed = 42 + worker_id
    np.random.seed(seed)
    random.seed(seed)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
device="cuda"
model.to(device)
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
encoded_input.to(device)
with torch.no_grad():
    model_output = model(**encoded_input)
    sentence_embeddings = model_output[0][:, 0]
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)

/usr/local/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Sentence embeddings: tensor([[-0.0401, -0.0034, -0.0373,  ..., -0.0112, -0.0444,  0.0459],
        [-0.0226,  0.0141, -0.0487,  ..., -0.0215, -0.0595,  0.0210]],
       device='cuda:0')


In [4]:
def getSim(q,p,tokenizer,model):
    encoded_input = tokenizer([q,p], padding=True, truncation=True, return_tensors='pt')
    encoded_input.to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    q_embeddings=sentence_embeddings[0]
    p_embeddings=sentence_embeddings[1]
    scores = q_embeddings @ p_embeddings.T
    return scores

In [5]:
import random
random.seed(4396)
random_int1 = random.randint(100, 149)
random_int2 = random.randint(100, 149)
random_int3 = random.randint(100, 149)
print(random_int1, random_int2, random_int3)

121 106 120


In [7]:
import json
import time
import random
def getDatalist(jsonPath):
    data_list = []
    with open(jsonPath, 'r') as input:
        for jsonObj in input:
            testObj = jsonObj.strip()
            data_list.append(json.loads(testObj))
        input.close()
    return data_list
train_neg_path="negetive_documents"
train_neg_json=getDatalist(train_neg_path)
train_pos_path="fake_positive_documents"
train_pos_json=getDatalist(train_pos_path)

def getNegPairs(datalist, idx):
    res=[]
    for i, item in enumerate(datalist):
        res.append({
            "id":i,
            "question": item["question"],
            "negative": [item["retrieval"][idx]["doc"],item["retrieval"][idx+1]["doc"],item["retrieval"][idx+2]["doc"]],
            "label": 0
        })
    return res

def getRandomNegPairs(datalist):
    random.seed(4396)
    random_int1 = random.randint(100, 149)
    random_int2 = random.randint(100, 149)
    random_int3 = random.randint(100, 149)
    res=[]
    for i, item in enumerate(datalist):
        res.append({
            "id":i,
            "question": item["question"],
            "negative": [item["retrieval"][random_int1]["doc"],item["retrieval"][random_int2]["doc"],item["retrieval"][random_int3]["doc"]],
            "label": 0
        })
    return res
train_negs=getRandomNegPairs(train_neg_json)

{'id': 1,
 'question': 'who makes bromocriptine\ni am wondering what company makes the drug bromocriptine, i need it for a mass i have on my pituitary gland and the cost just keeps raising. i cannot ever buy a full prescription because of the price and i was told if i get a hold of the maker of the drug sometimes they offer coupons or something to help me afford the medicine. if i buy 10 pills in which i have to take 2 times a day it costs me 78.00. and that is how i have to buy them.  thanks.',
 'negative': ["SUBJECT: My humerus bone\nMESSAGE: Hello. My name is [NAME], I am 33 years old, from Romania in Europe and I have a 6 inches difference between my left humerus bone and my right one. It all started when I was around 4 years old and I had a bone illness called: bone cyst. First it affected my right femur and then my left shoulder. The bones were very fragile in the mentioned areas and I had fractured them many times very easily. I had 3 surgeries at my shoulder and I healed up aft

In [8]:
def getTriTexts(negs, pos):
    res=[]
    for p in pos:
        idx = p["id"]
        for n in negs:
            if idx==n["id"]:
                res.append({
                    "id":idx,
                    "question":p["question"],
                    "positive":p["retrieval"][-1]["doc"],
                    "negative":n["negative"]
                })
    return res
train_set = getTriTexts(train_negs,train_pos_json)

In [10]:
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

class ContrastiveDataset(Dataset):
    def __init__(self, datalist, tokenizer):
        self.datalist=datalist
        self.tokenizer=tokenizer
    def __len__(self):
        return len(self.datalist)
    def __getitem__(self, idx):
        q_ids=self.tokenizer(self.datalist[idx]["question"], padding=True, truncation=True, return_tensors='pt')
        p_ids=self.tokenizer(self.datalist[idx]["positive"], padding=True, truncation=True, return_tensors='pt')
        n_ids=self.tokenizer(self.datalist[idx]["negative"], padding=True, truncation=True, return_tensors='pt')
        return {
            "q_id":q_ids,
            "p_id":p_ids,
            "n_id":n_ids,
            "question":self.datalist[idx]["question"],
            "positive":self.datalist[idx]["positive"],
            "negative":self.datalist[idx]["negative"],
        }
def contrastive_collate_fn(batch):
    questions = [item['question'] for item in batch]
    positives = [item['positive'] for item in batch]
    negatives = [item['negative'][0] for item in batch]
    q_inputs = [tokenizer(text, truncation=True, padding=False) for text in questions]
    p_inputs = [tokenizer(text, truncation=True, padding=False) for text in positives]
    n_inputs = [tokenizer(text, truncation=True, padding=False) for text in negatives]
    questions_padded = tokenizer.pad(q_inputs, padding=True, return_tensors='pt')
    positives_padded = tokenizer.pad(p_inputs, padding=True, return_tensors='pt')
    negatives_padded = tokenizer.pad(n_inputs, padding=True, return_tensors='pt')
    return {
        "question": questions_padded,
        "positive": positives_padded,
        "negative": negatives_padded,
    }
def contrastive_collate_fn(batch):
    questions = [item['question'] for item in batch]
    positives = [item['positive'] for item in batch]
    negatives = [item['negative'] for item in batch]
    q_inputs = [tokenizer(text, truncation=True, padding=False) for text in questions]
    p_inputs = [tokenizer(text, truncation=True, padding=False) for text in positives]
    questions_padded = tokenizer.pad(q_inputs, padding=True, return_tensors='pt')
    positives_padded = tokenizer.pad(p_inputs, padding=True, return_tensors='pt')
    flattened_negatives = [neg for neg_list in negatives for neg in neg_list]
    n_inputs = [tokenizer(text, truncation=True, padding=False) for text in flattened_negatives]
    negatives_padded = tokenizer.pad(n_inputs, padding=True, return_tensors='pt')
    return {
        "question": questions_padded,
        "positive": positives_padded,
        "negative": negatives_padded,
    }
train_dataset = ContrastiveDataset(train_set, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=contrastive_collate_fn, worker_init_fn=worker_init_fn)
first_batch = next(iter(train_dataloader))
print("First batch:", first_batch["negative"]["input_ids"].shape)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


First batch: torch.Size([3, 132])


In [11]:
print(getSim(train_negs[1]["question"],train_negs[1]["negative"][-1],tokenizer,model))
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
infonce_loss = InfoNCE(negative_mode='paired')
model.train()
batch_losses = []
for epoch in range(5):
    epoch_losses = [] 
    for batch in train_dataloader:
        q_ids = batch["question"].to(device)
        p_ids = batch["positive"].to(device)
        n_ids = batch["negative"].to(device)
        q_emb = model(**q_ids).last_hidden_state[:, 0, :]
        p_emb = model(**p_ids).last_hidden_state[:, 0, :]
        n_emb = model(**n_ids).last_hidden_state[:, 0, :]
        emb_size = n_emb.shape[-1]
        n_emb = n_emb.view(BATCH_SIZE, 3, emb_size)
        loss = infonce_loss(q_emb, p_emb, n_emb)
        epoch_losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    batch_losses.extend(epoch_losses)
    print(f"Epoch {epoch+1}: Loss = {sum(epoch_losses) / len(epoch_losses)}")
    print(getSim(train_negs[1]["question"],train_negs[1]["negative"][-1],tokenizer,model))

tensor(0.3373, device='cuda:0')
Epoch 1: Loss = 0.008785298461409638
tensor(-0.2993, device='cuda:0')
Epoch 2: Loss = 1.0914048927588737e-05
tensor(-0.3837, device='cuda:0')
Epoch 3: Loss = 0.031121241017581156
tensor(0.0064, device='cuda:0')
Epoch 4: Loss = 0.0006548297641243153
tensor(-0.3472, device='cuda:0')
Epoch 5: Loss = 2.431024562224593e-05
tensor(-0.3818, device='cuda:0')


# Inference

In [15]:
train_data = getDatalist("train_data")
val_data = getDatalist("val_data")
test_data = getDatalist("test_data")

In [16]:
def getSims(q, passages, tokenizer, model, device):
    texts = [q] + passages
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    encoded_input = encoded_input.to(device)

    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)

    q_embeddings = sentence_embeddings[0]
    p_embeddings = sentence_embeddings[1:]
    scores = torch.matmul(p_embeddings, q_embeddings.T)

    return scores

def find_best_passages(data, tokenizer, model, device, top_n):
    best_results = []
    for item in data:
        query = item["question"]
        summary = item["summary"]
        passages = [p["doc"] for p in item["retrieval"]]
        if top_n>0:
            passages = passages[:top_n]
        scores = getSims(query, passages, tokenizer, model, device)
        best_idx = torch.argmax(scores).item()
        best_passage = passages[best_idx]
        best_score = scores[best_idx].item()
        best_results.append({
            "id": item["id"],
            "question": query,
            "summary":summary,
            "retrieval": [{"doc":best_passage,"score":best_score}],
        })
        if len(best_results)%10==0:
            print(len(best_results))
    return best_results

In [22]:
val_rerank = find_best_passages(val_data, tokenizer, model, device, 20)
test_rerank = find_best_passages(test_data, tokenizer, model, device, 20)
train_rerank = find_best_passages(train_data, tokenizer, model, device, 20)

10
20
30
40
50
10
20
30
40
50
60
70
80
90
100
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000


In [1]:
def dicts2jsons(dicts, des_path):
    with open(des_path, 'w', encoding='utf-8') as file:
        for item in dicts:
            file.write(json.dumps(item)+"\n")
        file.close()
    return

In [20]:
train_json_path="train_json_path"
dicts2jsons(train_rerank, train_json_path)
test_json_path="test_json_path"
dicts2jsons(test_rerank, test_json_path)
val_json_path="val_json_path"
dicts2jsons(val_rerank, val_json_path)